<H1>Exploring the Austin Texas Area<H1>

<H2><H2>Libraries

In [ ]:
#Importing the libraries needed to get started. 
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium
!pip install lxml
!pip install uszipcode
!pip install beautifulsoup4


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import json # library to handle JSON files


import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import re
from geopy.geocoders import Nominatim
import pprint
import folium
from folium.features import DivIcon
from folium import IFrame
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.cluster import KMeans
from uszipcode import SearchEngine
from uszipcode import Zipcode
from pandas.io.json import json_normalize
import pandas as pd 
import numpy as np
import requests
print('Libraries imported.')

# Four Square credentials
client_id = 'P0MEEIQBEYLE2ZANW1HBPQCG150PIQA4OQUBPWBTQIUDDMXT'
client_secret = 'RGWBZJNJD2C12JE45VGZTRARXGUBKQS41YPBGI4U2NNLLEUE'
version = '20180605'

# Zillow ZWSID
zwsid = "X1-ZWz174hlm3pybv_2ox88"

# OpenCage Geocoding API key
my_api_key = "ffa7f2d7f52241e89950f04f26c095d2"

<H2><H2>Credentials for Foursquare zillow and opencage geocoding

In [ ]:
# Four Square credentials
client_id = 'P0MEEIQBEYLE2ZANW1HBPQCG150PIQA4OQUBPWBTQIUDDMXT'
client_secret = 'RGWBZJNJD2C12JE45VGZTRARXGUBKQS41YPBGI4U2NNLLEUE'
version = '20180605'

# Zillow ZWSID
zwsid = "X1-ZWz174hlm3pybv_2ox88"

# OpenCage Geocoding API key
my_api_key = "ffa7f2d7f52241e89950f04f26c095d2"

<H2><H2> Looking at the Austin area for most populated cities to explore.

In [ ]:
# Using wikipedia to get the city information. 
austin_nearby = "https://en.wikipedia.org/wiki/Greater_Austin"
response = requests.get(austin_nearby)

# BeautifulSoup to Parse
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find_all("table", class_="wikitable")
population_table = table[1]
pop_def = population_table.find_all('td')

city_list = []
population_list = []

#looking for the top 5:
for i in range(0, 16, 3):
    city_list.append(pop_def[i].text.strip())
    population_list.append(pop_def[i+1].contents[0])

city_df = pd.DataFrame(columns=["CityName", "Population"])
city_df["CityName"] = city_list 
city_df["Population"] = population_list
city_df

<H3> Top 3 cities with latitude and longitude using opencage

In [ ]:
# Top 3 with latitude and Longitude.
city_list = ['Austin', 'Round Rock', 'Georgetown']
latitude_list = []
longitude_list = []
top_city_df = pd.DataFrame(columns=["CityName", "Population", "Latitude", "Longitude"])

for city in city_list:
    query = city + ", TX, United States of America"
    uri = "https://api.opencagedata.com/geocode/v1/json?q={}&key={}".format(query, my_api_key)
    response = requests.get(uri).json()
    latitude_list.append(response['results'][0]['geometry']['lat'])
    longitude_list.append(response['results'][0]['geometry']['lng'])
    
top_city_df["CityName"] = city_list
top_city_df["Latitude"] = latitude_list
top_city_df["Population"] = city_df["Population"]
top_city_df["Longitude"] = longitude_list
    
top_city_df


<H3> Median home value for all three cities covering 2019, 2020 and forecast for 2021.

In [ ]:
# Using payload for the analysis.
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

median_values = []
last_change = []
forecast = []

cities = ["Austin", "Round-rock", "Georgetown"]
for city in cities:
    url = "https://www.zillow.com/{}-tx/home-values/".format(city)
    response = requests.get(url, headers=req_headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    median = soup.find_all('h2')
    median_values.append(median[0].text)
    
    change_list = soup.find_all('ul', class_='zsg-g')
    list_item = change_list[0].find_all('li')
    last_change.append(list_item[0].text)    
    forecast.append(list_item[1].text.strip())
    
home_value_dataframe = pd.DataFrame(columns=["City", "Last Year Value", "Current Median Value", "Next Year Forecast"])
home_value_dataframe["City"] = cities
home_value_dataframe["Last Year Value"] = last_change
home_value_dataframe["Current Median Value"] = median_values
home_value_dataframe["Next Year Forecast"] = forecast

home_value_dataframe

In [ ]:
# Need to create two lists to parse.

new_ly_values = []
new_fc_values = []

# regex pattern to clean up the data
percentage_regex = re.compile(r'\d+.\d+%')

# parsing through to match regex and adding it to the lists mentioned above.
for values in home_value_dataframe['Last Year Value']:
    sliced_value = percentage_regex.search(values)
    new_ly_values.append(sliced_value.group())

for values in home_value_dataframe['Next Year Forecast']:
    sliced_value = percentage_regex.search(values)
    new_fc_values.append(sliced_value.group())

home_value_dataframe["Last Year Value"] = new_ly_values
home_value_dataframe["Next Year Forecast"] = new_fc_values
home_value_dataframe['Current Median Value'] = home_value_dataframe['Current Median Value'].str.replace("$", "")
home_value_dataframe['Current Median Value'] = home_value_dataframe['Current Median Value'].str.replace(",", "")


home_value_dataframe

In [ ]:
home_value_dataframe['Last Year Value'] = home_value_dataframe['Last Year Value'].str.replace("%","")
home_value_dataframe['Next Year Forecast'] = home_value_dataframe['Next Year Forecast'].str.replace("%","")

for i in range(len(home_value_dataframe)):
    last_percent = float(home_value_dataframe.loc[i, 'Last Year Value'])
    forecast = float(home_value_dataframe.loc[i, 'Next Year Forecast'])
    median_value = float(home_value_dataframe.loc[i, 'Current Median Value'])
    
    last = float(last_percent/100)
    fore = float(forecast/100)
    home_value_dataframe.loc[i, 'Last Year Value'] = (1 - last) * median_value
    home_value_dataframe.loc[i, 'Next Year Forecast'] = (1 + fore) * median_value


home_value_dataframe

<H3>Providing one consoidated view of the findings using the methodology and data.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('classic')
import numpy as np
from pandas import Series, DataFrame
 
# plotting the health venues in three cities for easier comparison
data = [401999, 380291, 79604]
labels = ["Austin","Round Rock", "Georgetown"]
width = .25
plt.xticks(range(len(data)), labels, fontsize=14)
plt.xlabel('City',fontsize=14)
plt.ylabel('Median Price')
plt.title("Current Median comparison")
plt.xticks(range(len(data)), labels)
plt.bar(range(len(data)), data, width=width, color= 'blue') 

plt.show() 


<H3> Providing three individual views for a comparison within each city.

In [ ]:
# Austin Median Compare
import matplotlib.pyplot as plt
plt.style.use('classic')
import numpy as np
from pandas import Series, DataFrame
  

plt.rcParams['axes.facecolor']='lightblue'
plt.rcParams['savefig.facecolor']='lightblue'


x = [2019, 2020, 2021]
y = [380291,401990,404810]

my_xticks = ['2019', '2020', '2021']
plt.figure(figsize=(7,6))

plt.xticks(x, my_xticks, fontsize=14)
plt.yticks(fontsize=12)

plt.title("Austin Median Price 2019-2021", fontsize=16, color = 'black')
plt.plot(x , y, 'b-o',color ='red')
for a, b in zip(x, y):
    val = "$" + str(b)
    plt.text(a+0.08, b, val, fontsize=15, color='red' )
        

In [ ]:
# Round rock median compare
import matplotlib.pyplot as plt
plt.style.use('classic')
import numpy as np
from pandas import Series, DataFrame
  

plt.rcParams['axes.facecolor']='lightblue'
plt.rcParams['savefig.facecolor']='lightblue'


x = [2019, 2020, 2021]
y = [299162, 302184,312458]

my_xticks = ['2019', '2020', '2021']
plt.figure(figsize=(7,6))

plt.xticks(x, my_xticks, fontsize=14)
plt.yticks(fontsize=12)

plt.title("Round Rock Median Price 2019-2021", fontsize=16, color = 'black')
plt.plot(x , y, 'b-o',color ='red')
for a, b in zip(x, y):
    val = "$" + str(b)
    plt.text(a+0.08, b, val, fontsize=15, color='red' )

In [ ]:
# Georgetown median compare
import matplotlib.pyplot as plt
plt.style.use('classic')
import numpy as np
from pandas import Series, DataFrame
  
x = [2019, 2020, 2021]
y = [299162, 302184,312458]
plt.rcParams['axes.facecolor']='lightblue'
plt.rcParams['savefig.facecolor']='lightblue'

my_xticks = ['2019', '2020', '2021']
plt.figure(figsize=(7,6))

plt.xticks(x, my_xticks, fontsize=12)
plt.yticks(fontsize=12)
plt.title("Georgetown Median Price 2019-2021", fontsize=16, color = 'black')
plt.plot(x , y, 'b-o',color ='red')
for a, b in zip(x, y):
    val = "$" + str(b)
    plt.text(a+0.08, b, val, fontsize=15, color='red' )

<H2> Exploring the housing prices based on median values

<H3>Looking for the median household income leveraging zipmaps 

In [ ]:
# Getting zip codes of austin for the search

austin_zip = "http://www.city-data.com/zipmaps/Austin-Texas.html"
response = requests.get(austin_zip)
ziplist = []
# Parsing the results with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find_all('div', class_="zip data-block")
for t in table:
    ziplist.append(t["id"])
    
# sample ziplist of Austin
ziplist[0:10]

In [ ]:
# cleaning the dataframe 
# setting the search engine
search = SearchEngine(simple_zipcode=True)
results = []
for z in ziplist:
    zipcode = search.by_zipcode(z)
    data = zipcode.to_dict()
    results.append(data)

# storing the results in a dataframe    
austin_zip_df = json_normalize(results)
austin_zip_df.head()

In [ ]:
# narrowing columns for analysis
austin_zip_df = austin_zip_df.drop(['bounds_east', 'bounds_north', 'bounds_south', 'area_code_list', 'common_city_list', 'population_density', 'bounds_west', 'major_city', 'post_office_city', 'state', 'timezone', 'zipcode_type'], 1)
austin_zip_df.head()

In [ ]:
# cleaning the dataframe a bit to make it more readable and usable
austin_zip_df['county'] = austin_zip_df['county'].str.replace("County","")
zipcol = austin_zip_df['zipcode']
austin_zip_df.drop('zipcode', axis=1,inplace=True)
austin_zip_df.insert(0, 'zipcode', zipcol)

austin_zip_df.head()

In [ ]:
# one-hot encoding the county names
county_dummies = pd.get_dummies(austin_zip_df['county'], prefix='county')
austin_df = pd.concat([austin_zip_df, county_dummies], axis=1)
austin_df = austin_df.drop('county', 1)
austin_df.head()

In [ ]:
# set the number of clusters
k = 3

# running K-Means clustering
kmeans = KMeans(n_init=400, n_clusters=k, random_state=1).fit(austin_df)
kmeans.labels_

# adding the cluster labels to the dataframe
austin_df.insert(0, 'Cluster', kmeans.labels_)
austin_df.head()

In [ ]:
# examining each cluster
cluster_one = austin_df[austin_df.Cluster == 0]
cluster_one.sort_values("median_home_value", axis = 0, ascending = True, inplace = True) 
cluster_one

In [ ]:
cluster_two = austin_df[austin_df.Cluster == 1]
cluster_two.sort_values("median_home_value", axis = 0, ascending = True, inplace = True) 
cluster_two

In [ ]:
cluster_three = austin_df[austin_df.Cluster == 2]
cluster_three.sort_values("median_home_value", axis = 0, ascending = True, inplace = True) 
cluster_three

<H2> Home prices in the above clusters provide buyers insight into home pricing:

<H4>Cluster One = 72k-237k
<H4>Cluster Two = 247-442k
<H4>Cluster Three = 460k-641k

<H3>Analysis provides answer to those that have a price range in which they are seeking to stay within. 

### Leveraging FourSquare's API to explore Austin, Round Rock and Georgetown.

In [ ]:
# Four Square credentials
client_id = 'P0MEEIQBEYLE2ZANW1HBPQCG150PIQA4OQUBPWBTQIUDDMXT'
client_secret = 'RGWBZJNJD2C12JE45VGZTRARXGUBKQS41YPBGI4U2NNLLEUE'
version = '20180605'

In [ ]:
# defining a class to to explore venues under various categories
def requestPayload(latitude, longitude, category):
    # crafting the request url
    limit = 500      # no. of locations to return
    radius = 8050    # approx 5 miles
    url = "https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&v={}&limit={}&radius={}&ll={},{}".format(category, client_id, client_secret, version, limit, radius, latitude, longitude)
    return url 

### Leveraging categories in foursquare to provide schools in each area.

#### 1. Schools

In [ ]:
# Category list that with category IDs of different schools
schools_list = [
    "4d4b7105d754a06372d81259", "4f4533804b9074f6e4fb0105", "4f4533814b9074f6e4fb0106", 
    "4bf58dd8d48988d13d941735", "52e81612bcbc57f1066b7a45"]


# creating an empty dataframe to store the results
schools_dataframe = pd.DataFrame(columns=["CityName", "Venue", "Latitude", "Longitude"])
venues_list = []
category_list =[]
category_type =[]
latitude_list = []
longitude_list = []
city_list = []

for city in range(3):
    for category in schools_list:
        response = requests.get(requestPayload(top_city_df.loc[city, "Latitude"], top_city_df.loc[city, "Longitude"], category)).json()
        for i in range(len(response['response']['venues'])):
            category_list.append(response['response']['venues'][i]['name'])
            category_type.append(response['response']['venues'][i]['name'])
            venues_list.append(response['response']['venues'][i]['name'])
            latitude_list.append(response['response']['venues'][i]['location']['lat'])
            longitude_list.append(response['response']['venues'][i]['location']['lng'])
            city_list.append(top_city_df.loc[city, "CityName"])
            
schools_dataframe["CityName"] = city_list
schools_dataframe["Venue"] = venues_list
schools_dataframe["category"]= category_list
schools_dataframe["cattype"]= category_type
schools_dataframe["Latitude"] = latitude_list
schools_dataframe["Longitude"] = longitude_list

In [ ]:
schools_dataframe.to_csv(r'c:\users\msanford\schools.csv')

In [ ]:
# Schools 

ax = schools_dataframe['CityName'].value_counts().plot(kind='bar', figsize=(8,6), fontsize=12, color='purple')
ax.set_xlabel("City", fontsize=14)
plt.xticks(rotation=0)
ax.set_ylabel("# of Schools", fontsize=14)
plt.title("Schools in Austin, Round Rock & Georgetown", fontsize=15)

for i in ax.patches:
    ax.text(i.get_x()+.17, i.get_height()+0.6, str(i.get_height()), fontsize=14)



#### Leveraging categories in foursquare to provide schools available.

#### Grouping the categories

In [ ]:
summary = schools_dataframe.groupby('CityName').count().reset_index()
summary['Count'] = summary['Venue']
summary = summary.drop(['Latitude', 'Longitude', 'Venue', 'category'], axis=1)
summary = summary.sort_values('Count').reset_index(drop=True)
summary.head()

#### 2. Food

In [ ]:
# Food
food_list = [
    "4d4b7105d754a06374d81259", "4bf58dd8d48988d14e941735", "4bf58dd8d48988d142941735", "4bf58dd8d48988d111941735", 
"4bf58dd8d48988d113941735", "4eb1d5724b900d56c88a45fe", "4bf58dd8d48988d1d1941735", "4bf58dd8d48988d149941735", 
"4bf58dd8d48988d1df931735", "4bf58dd8d48988d143941735", "4bf58dd8d48988d16d941735", "4bf58dd8d48988d1e0931735", 
 "4bf58dd8d48988d147941735", "4bf58dd8d48988d16e941735"]

# creating an empty dataframe to store the results
food_dataframe = pd.DataFrame(columns=["CityName", "Venue", "Latitude", "Longitude"])
venues_list = []
category_list =[]
category_type=[]
latitude_list = []
longitude_list = []
city_list = []

for city in range(3):
    for category in food_list:
        response = requests.get(requestPayload(top_city_df.loc[city, "Latitude"], top_city_df.loc[city, "Longitude"], category)).json()
        for i in range(len(response['response']['venues'])):
            venues_list.append(response['response']['venues'][i]['name'])
            category_list.append(response['response']['venues'][i]['name'])
            category_type.append(response['response']['venues'][i]['name'])
            latitude_list.append(response['response']['venues'][i]['location']['lat'])
            longitude_list.append(response['response']['venues'][i]['location']['lng'])
            city_list.append(top_city_df.loc[city, "CityName"])
            
food_dataframe["CityName"] = city_list
food_dataframe["Venue"] = venues_list
food_dataframe["category"]= category_list
food_dataframe["Latitude"] = latitude_list
food_dataframe["cattype"]= category_type
food_dataframe["Longitude"] = longitude_list


In [ ]:
# Food
food_list = [
    "4d4b7105d754a06374d81259", "4bf58dd8d48988d14e941735", "4bf58dd8d48988d142941735", "4bf58dd8d48988d111941735", 
"4bf58dd8d48988d113941735", "4eb1d5724b900d56c88a45fe", "4bf58dd8d48988d1d1941735", "4bf58dd8d48988d149941735", 
"4bf58dd8d48988d1df931735", "4bf58dd8d48988d143941735", "4bf58dd8d48988d16d941735", "4bf58dd8d48988d1e0931735", 
 "4bf58dd8d48988d147941735", "4bf58dd8d48988d16e941735"]

# creating an empty dataframe to store the results
food_dataframe = pd.DataFrame(columns=["CityName", "Venue", "Latitude", "Longitude"])
venues_list = []
category_list =[]
category_type=[]
latitude_list = []
longitude_list = []
city_list = []

for city in range(3):
    for category in food_list:
        response = requests.get(requestPayload(top_city_df.loc[city, "Latitude"], top_city_df.loc[city, "Longitude"], category)).json()
        for i in range(len(response['response']['venues'])):
            venues_list.append(response['response']['venues'][i]['name'])
            category_list.append(response['response']['venues'][i]['name'])
            category_type.append(response['response']['venues'][i]['name'])
            latitude_list.append(response['response']['venues'][i]['location']['lat'])
            longitude_list.append(response['response']['venues'][i]['location']['lng'])
            city_list.append(top_city_df.loc[city, "CityName"])
            
food_dataframe["CityName"] = city_list
food_dataframe["Venue"] = venues_list
food_dataframe["category"]= category_list
food_dataframe["Latitude"] = latitude_list
food_dataframe["cattype"]= category_type
food_dataframe["Longitude"] = longitude_list


In [ ]:
food_dataframe.to_csv(r'c:\users\msanford\food.csv')

In [ ]:
# Food venues 
ax = food_dataframe['CityName'].value_counts().plot(kind='bar', figsize=(8,6), fontsize=12, color='pink')
ax.set_xlabel("City", fontsize=14)
plt.xticks(rotation=0)
ax.set_ylabel("# of Venues", fontsize=12)
plt.title("Food Venues in Austin, Round Rock & Georgetown", fontsize=15)

for i in ax.patches:
    ax.text(i.get_x()+.17, i.get_height()+4.0, str(i.get_height()), fontsize=14)

### Leveraging categories in foursquare to provide food vendues available.

### Grouping for count

In [ ]:
summary = food_dataframe.groupby('CityName').count().reset_index()
summary['Count'] = summary['Venue']
summary = summary.drop(['Latitude', 'Longitude', 'Venue', 'category'], axis=1)
summary = summary.sort_values('Count').reset_index(drop=True)
summary.head()

<H2> Map of the food and school venues in Austin

In [ ]:
# Combining all different venue categories of Austin into a single dataframe
austin_dataframe = pd.concat([
    
    food_dataframe[food_dataframe.CityName == "Austin"],
    schools_dataframe[schools_dataframe.CityName == "Austin"]],ignore_index=True)

print(austin_dataframe.shape)

In [ ]:
# Map of all these Austin venues into a map
austin_map = folium.Map(location=[30.271129, -97.743700], zoom_start=11)

# adding markers
for lat, lng, venue in zip(austin_dataframe['Latitude'], austin_dataframe['Longitude'], austin_dataframe['Venue']):
    label = "".format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(austin_map)

# Austin with all category venues plotted
austin_map

<H2>Results and Conclusion

<H5>Austin and the surrounding areas, the three cities, Austin, Round Rock, Georgetown were the most populated.
<H5>Austin has a very good pricing range and is a good choice for a home buyer. 
<H5>This result was proven with the number of schools, food venues, home values and forecasting analysis and data.  
<H5>The projected forecast for the median home in Austin was clearly proved to be on the rise in the next year.  
<H5>The clustering and grouping of the data sets and methodologies noted led to some insight that is supported by the data itself.   

<H2>It’s a great spot to call home and work remotely!  
